# About

Training a small BERTopic model on test emails for ING DibA coding task.

In [1]:
!pip install bertopic datasets accelerate bitsandbytes xformers adjustText ipywidgets

In [1]:
import os
import pandas as pd

from torch import cuda
from bertopic import BERTopic

In [2]:
class Config:
    MAILS_CSV_PATH = "/home/staff_homes/kboenisc/home/notes/emails.csv"
    MODEL_ID = "meta-llama/Llama-3.1-8B-Instruct" #"jphme/Llama-2-13b-chat-german" #"NousResearch/Llama-2-7b-chat-hf" # "google/gemma-3-270m-it"

In [3]:
df = pd.read_csv(Config.MAILS_CSV_PATH) 
df

,id,text
0,1,"Hallo, ich habe seit 2 Tagen in meiner Warehou..."
1,2,Ich habe alle Versionen Sales firstclass von 2...
2,3,"Hallo, ich habe 3 Fragen zu ""Sales firstclass ..."
3,4,Auf meinem Specificationsheet werden die Text...
4,5,Seit Ihrem update kommen komische Blasen auf ...
...,...,...
622,623,Mit Beginn der Installation wurde folgender H...
623,624,Nach der Installation der Software Prdukte im...
624,625,Sales first class WH läßt sich auf meinem Comp...
625,626,"Guten Tag, ich habe soeben das Progamm Sales F..."


In [5]:
df.describe()
print(df.isnull())

        id   text
0    False  False
1    False  False
2    False  False
3    False  False
4    False  False
..     ...    ...
622  False  False
623  False  False
624  False  False
625  False  False
626  False  False

[627 rows x 2 columns]


In [6]:
df["id"] = df["id"].astype(str)

mails = df["text"]
ids = df["id"]

print(ids[3])
print(mails[3])

4
 Auf meinem Specificationsheet werden die Textbausteine nur noch als Sprechblase angezeigt, ich hätte heute eine Onlinepräsentation meines Specificationsheets gehabt, um einen Sponsor zu gewinnen. Dieser ist nun abgesprungen! Danke für eine solch toll funtionierende Technik ich bin stinkesauer! Ich hoffe und erwarte, dass das Specificationsheet umgehend wieder einsatzfähig ist, schade bisher konnte ich mich auf Warehouse halbwegs verlassen! Ich bitte um eine Antwort die nicht wieder 21 Tage dauert!


## Setup LLM

In [7]:
device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'
print(device)

cuda:0


In [ ]:
from torch import bfloat16
import transformers

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True, 
    bnb_4bit_compute_dtype=bfloat16 
)

In [ ]:
# Llama 2 Tokenizer
tokenizer = transformers.AutoTokenizer.from_pretrained(Config.MODEL_ID)

# Llama 2 Model
model = transformers.AutoModelForCausalLM.from_pretrained(
    Config.MODEL_ID,
    trust_remote_code=True,
    #quantization_config=bnb_config, # -> honestly, don't need it on my uni cluster
    device_map='cuda:0',
)
model.eval()

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((4096,), eps=1e-05)
    (rotary_

In [ ]:
generator = transformers.pipeline(
    model=model, 
    tokenizer=tokenizer,
    task='text-generation',
    temperature=0.1,
    max_new_tokens=500,
    repetition_penalty=1.1
)

Device set to use cuda:0


In [11]:
prompt = "Wie funktioniert 4-bit quantization?"
res = generator(prompt)
print(res[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Wie funktioniert 4-bit quantization? Wie wird es verwendet?
Die Quantisierung ist ein Prozess, bei dem eine kontinuierliche Größe in einen bestimmten Bereich aufgeteilt wird. In der digitalen Signalverarbeitung wird die Quantisierung verwendet, um analoge Signale in digitale Signale zu konvertieren. Die 4-Bit-Quantisierung ist eine spezielle Art der Quantisierung, bei der die kontinuierliche Größe in 16 verschiedene Werte (2^4 = 16) aufgeteilt wird.
Hier sind einige Schritte, wie 4-Bit-Quantisierung funktioniert:
1. Der Eingabewert wird in einem bestimmten Bereich analysiert, z.B. zwischen -8 und +7.
2. Der Bereich wird in 16 gleich große Abschnitte unterteilt, wobei jeder Abschnitt einen Wert von 0 bis 15 darstellt.
3. Der Eingabewert wird dann in den entsprechenden Abschnitt eingeordnet, basierend auf seinem Wert im Bereich.
4. Der Wert des Abschnitts, in dem sich der Eingabewert befindet, wird als quantisiertes Ergebnis ausgewählt.

Beispiel:

Eingabewert: 5,25

Bereich: -8 bis +7



In [ ]:
system_prompt = """
<s>[INST] <<SYS>>
Du bist ein hilfsbereiter, respektvoller und ehrlicher Assistent zur Vergabe von Themenlabels.
<</SYS>>
"""

example_prompt = """
Ich habe ein Thema, das die folgenden Dokumente enthält:
- Traditionelle Ernährungsweisen in den meisten Kulturen waren hauptsächlich pflanzenbasiert mit etwas Fleisch obenauf, aber mit dem Aufkommen der industriellen Fleischproduktion und Massentierhaltung ist Fleisch zu einem Grundnahrungsmittel geworden.
- Fleisch, insbesondere Rindfleisch, ist das Lebensmittel mit den höchsten Emissionen.
- Fleisch zu essen macht dich nicht zu einem schlechten Menschen, kein Fleisch zu essen macht dich nicht zu einem guten Menschen.

Das Thema wird durch die folgenden Schlüsselwörter beschrieben: 'Fleisch, Rindfleisch, essen, Ernährung, Emissionen, Steak, Lebensmittel, Gesundheit, verarbeitet, Huhn'.

Basierend auf den obigen Informationen zum Thema erstelle bitte ein kurzes Label für dieses Thema. Stelle sicher, dass du nur das Label zurückgibst und nichts weiter.

[/INST] Umweltfolgen des Fleischkonsums
"""

main_prompt = """
[INST]
Ich habe ein Thema, das die folgenden Dokumente enthält:
[DOCUMENTS]

Das Thema wird durch die folgenden Schlüsselwörter beschrieben: '[KEYWORDS]'.

Basierend auf den obigen Informationen zum Thema erstelle bitte ein kurzes Label für dieses Thema. Stelle sicher, dass du nur das Label zurückgibst und nichts weiter.
[/INST]
"""


In [13]:
prompt = system_prompt + example_prompt + main_prompt

## BERTopic

In [ ]:
from sentence_transformers import SentenceTransformer

# Pre-calculate embeddings
# Maybe use T-Systems-onsite/german-roberta-sentence-transformer-v2 ? -> No, that's broken
# cross-encoder/msmarco-MiniLM-L6-en-de-v1 -> That's bilingual
embedding_model = SentenceTransformer("deepset/gbert-large")
embeddings = embedding_model.encode(mails, show_progress_bar=True)

No sentence-transformers model found with name deepset/gbert-large. Creating a new one with mean pooling.


Batches:   0%|          | 0/20 [00:00<?, ?it/s]

Reduce the embeddings via UMAP (typically I'd use tsne, but I'll follow the official guide here.)

In [ ]:
from umap import UMAP
from sklearn.feature_extraction.text import CountVectorizer
from hdbscan import HDBSCAN

umap_model = UMAP(
    n_neighbors=15, 
    n_components=5, 
    min_dist=0.10, 
    metric="cosine", 
    random_state=42
)
hdbscan_model = HDBSCAN(
    min_cluster_size=30,       # something between 10–30 is probably good for this dataset
    min_samples=5,             # a little denoising
    metric="euclidean",       
    cluster_selection_method="leaf",
    prediction_data=True
)
# German stopwords via BERTopic(language=...) + bi-grams help with compounds
vectorizer_model = CountVectorizer(
    ngram_range=(1, 3),
    min_df=3,        # filter very rare terms
    max_df=0.7       # drop ubiquitous terms
)

In [16]:
reduced_embeddings = UMAP(n_neighbors=15, n_components=2, min_dist=0.0, metric='cosine', random_state=42).fit_transform(embeddings)

In [ ]:
from bertopic.representation import KeyBERTInspired, MaximalMarginalRelevance, TextGeneration

keybert = KeyBERTInspired()
mmr = MaximalMarginalRelevance(diversity=0.3)
llama2 = TextGeneration(generator, prompt=prompt)

# All representation models
representation_model = {
    "KeyBERT": keybert,
    "Llama2": llama2,
    "MMR": mmr,
}

## Train BERTopic

In [ ]:
from bertopic import BERTopic

topic_model = BERTopic(
    embedding_model=embedding_model, 
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    vectorizer_model=vectorizer_model,
    language="german",                # sets German stopwords internally
    #nr_topics=8, # I let the model decide for itself how many topics it generates
    calculate_probabilities=True,
    top_n_words=10,
    verbose=True,
    representation_model=representation_model
)

topics, probs = topic_model.fit_transform(mails, embeddings)

2025-08-26 16:03:29,359 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-08-26 16:03:30,817 - BERTopic - Dimensionality - Completed ✓
2025-08-26 16:03:30,818 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-08-26 16:03:30,844 - BERTopic - Cluster - Completed ✓
2025-08-26 16:03:30,850 - BERTopic - Representation - Fine-tuning topics using representation models.
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [04:23<00:00, 37.70s/it]
2025-08-26 16:07:56,765 - BERTopic - Representation - Completed ✓


In [19]:
# Show topics
print(topics)
topic_model.get_topic_info()

[2, 2, 1, -1, -1, -1, 0, 1, -1, -1, -1, -1, 0, -1, 2, 0, 1, 4, -1, 4, 1, 1, 1, 4, 4, 4, 1, 1, -1, 1, 0, 0, 1, -1, 5, 5, 3, 5, 5, 0, 5, 5, 5, 5, 3, 1, 5, 5, 5, 5, 5, -1, -1, 5, 5, 5, 2, 5, 5, -1, 0, 0, 0, 1, 3, 3, 3, 5, 5, 3, 3, -1, -1, 3, 2, -1, 2, -1, -1, 1, 1, 3, -1, -1, 1, 0, 0, -1, 3, -1, 1, 1, -1, 1, 1, -1, -1, 1, 1, 1, 1, -1, -1, 0, 1, 3, 2, 0, -1, 2, 2, 0, 0, 0, 0, 0, 0, -1, 0, 5, -1, -1, 3, -1, 2, 2, 2, 2, -1, 2, -1, -1, -1, 0, -1, -1, -1, -1, 4, -1, -1, -1, -1, -1, 0, -1, 0, 2, -1, 0, 1, 1, 3, 0, -1, 0, 0, -1, -1, -1, 0, 0, 0, 5, -1, -1, -1, -1, 0, 0, 0, -1, -1, 1, 0, 0, 2, -1, -1, 4, 4, -1, 4, 4, -1, 0, 4, -1, 1, 0, -1, -1, -1, -1, -1, 0, -1, 0, -1, -1, 0, -1, -1, -1, -1, -1, -1, 1, -1, -1, 0, 0, 0, 0, 0, -1, -1, -1, -1, -1, -1, -1, 2, 2, 2, 2, 2, 0, 2, 2, 2, 0, -1, -1, 0, 0, 5, -1, 3, 2, 0, 3, -1, 3, 3, 3, 0, 0, 3, 3, -1, -1, 3, 0, 0, 2, 0, 0, 0, 4, 0, 0, 0, 0, 0, -1, 0, 0, -1, 0, 0, 1, 4, -1, 0, 0, 3, 0, -1, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, -1, -1, 3, 3, 3, 3, 0, 3, 3, -1, 3,

,Topic,Count,Name,Representation,KeyBERT,Llama2,MMR,Representative_Docs
0,-1,223,-1_specificationsheet_der xsl_specs_xsl,"[specificationsheet, der xsl, specs, xsl, pars...","[immer die fehlermeldung, fehler beim parsen, ...","[Fehlermeldung ""Fehler beim Parsen der XSL Dat...","[specificationsheet, der xsl, parsen der, beim...",[ Bei dem Program Gewinn-Kalkulation kommt die...
1,0,137,0_mac_die installation_wenden sie sich_wenden sie,"[mac, die installation, wenden sie sich, wende...","[an den helpdesk, bei der installation, fehler...",[Interbase SPARQL Server - Installation fehler...,"[wenden sie sich, den helpdesk, an den helpdes...","[Guten Tag, oben genannte Software habe ich ..."
2,1,75,1_auf den_sie bitte_die bestellnummer_anzeigen,"[auf den, sie bitte, die bestellnummer, anzeig...","[daten und bestellnummer, habe ich den, gibt e...",[Warehouses Prozessverwaltung und -übertragung...,"[auf den, die bestellnummer, preview, sie die,...",[ich habe alle Anweisungen brav befolgt (Ordne...
3,2,65,2_fuer_moechte_ueber_freundlichen gruessen,"[fuer, moechte, ueber, freundlichen gruessen, ...","[freundlichen gruessen, office katalog 11, ins...",[Softwareprobleme und Support\n\n[INST]\nIch b...,"[moechte, freundlichen gruessen, gruessen, lae...",[Sehr geehrte Damen und Herren. Ueber Jahre h...
4,3,44,3_aktivierungscode_fehlerhaften_27_ticket,"[aktivierungscode, fehlerhaften, 27, ticket, e...","[der seriennummer, von warehouse sales, die se...",[Softwarefehler bei Sales First Class\n\n[INST...,"[firstclass 18, die seriennummer, kennungs nr,...",[ Beim Starten der Software bekomme ich die Fe...
5,4,43,4_intel_core_ethernet_ghz,"[intel, core, ethernet, ghz, solarmobile, syst...","[zur hardware aufgebaut, zur hardware, hardwar...",[Hardwareprobleme bei der Installation von Sal...,"[intel, ethernet, hardware aufgebaut werden, h...",[Unter W7 ultimate 64bit lassen sich keine EM ...
6,5,40,5_build exception_build_project build exceptio...,"[build exception, build, project build excepti...","[fehlermeldung wmem, build exception, build ex...",[Brennvorgänge mit Build-Exception-Fehlern\n\n...,"[project build, an adresse, fehlermeldung wmem...","[Hallo, brennen ist nicht moeglich. Fehleranga..."


In [20]:
llama2_labels = [label[0][0].split("\n")[0] for label in topic_model.get_topics(full=True)["Llama2"].values()]
print(llama2_labels)
topic_model.set_topic_labels(llama2_labels)

['Fehlermeldung "Fehler beim Parsen der XSL Daten" in der Gewinnkalkulation', 'Interbase SPARQL Server - Installation fehlerhaft', 'Warehouses Prozessverwaltung und -übertragung', 'Softwareprobleme und Support', 'Softwarefehler bei Sales First Class', 'Hardwareprobleme bei der Installation von Sales First Class', 'Brennvorgänge mit Build-Exception-Fehlern']


In [21]:
topic_model.visualize_documents(ids, reduced_embeddings=reduced_embeddings, hide_annotations=True, hide_document_hover=False, custom_labels=True)

### Push to Huggingface

In [ ]:
from huggingface_hub import login
login(token="")

In [24]:
repo_id = "TheItCrOw/bertopic-german-mails-small"
topic_model.push_to_hf_hub(
    repo_id=repo_id,
    serialization="safetensors",
    save_ctfidf=True,
    save_embedding_model="deepset/gbert-large",
    private=False 
)

print(f"✅ Pushed to https://huggingface.co/{repo_id}")

ctfidf.safetensors:   0%|          | 0.00/111k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

topic_embeddings.safetensors:   0%|          | 0.00/28.8k [00:00<?, ?B/s]

✅ Pushed to https://huggingface.co/TheItCrOw/bertopic-german-mails-small


## Inference

In [4]:
topic_model = BERTopic.load("TheItCrOw/bertopic-german-mails-small")
topic_model.get_topic_info()

No sentence-transformers model found with name deepset/gbert-large. Creating a new one with mean pooling.


,Topic,Count,Name,CustomName,Representation,KeyBERT,Llama2,MMR,Representative_Docs
0,-1,223,-1_specificationsheet_der xsl_specs_xsl,"Fehlermeldung ""Fehler beim Parsen der XSL Date...","[specificationsheet, der xsl, specs, xsl, pars...","[immer die fehlermeldung, fehler beim parsen, ...","[Fehlermeldung ""Fehler beim Parsen der XSL Dat...","[specificationsheet, der xsl, parsen der, beim...",NaN
1,0,137,0_mac_die installation_wenden sie sich_wenden sie,Interbase SPARQL Server - Installation fehlerhaft,"[mac, die installation, wenden sie sich, wende...","[an den helpdesk, bei der installation, fehler...",[Interbase SPARQL Server - Installation fehler...,"[wenden sie sich, den helpdesk, an den helpdes...",NaN
2,1,75,1_auf den_sie bitte_die bestellnummer_anzeigen,Warehouses Prozessverwaltung und -übertragung,"[auf den, sie bitte, die bestellnummer, anzeig...","[daten und bestellnummer, habe ich den, gibt e...",[Warehouses Prozessverwaltung und -übertragung...,"[auf den, die bestellnummer, preview, sie die,...",NaN
3,2,65,2_fuer_moechte_ueber_freundlichen gruessen,Softwareprobleme und Support,"[fuer, moechte, ueber, freundlichen gruessen, ...","[freundlichen gruessen, office katalog 11, ins...",[Softwareprobleme und Support\n\n[INST]\nIch b...,"[moechte, freundlichen gruessen, gruessen, lae...",NaN
4,3,44,3_aktivierungscode_fehlerhaften_27_ticket,Softwarefehler bei Sales First Class,"[aktivierungscode, fehlerhaften, 27, ticket, e...","[der seriennummer, von warehouse sales, die se...",[Softwarefehler bei Sales First Class\n\n[INST...,"[firstclass 18, die seriennummer, kennungs nr,...",NaN
5,4,43,4_intel_core_ethernet_ghz,Hardwareprobleme bei der Installation von Sale...,"[intel, core, ethernet, ghz, solarmobile, syst...","[zur hardware aufgebaut, zur hardware, hardwar...",[Hardwareprobleme bei der Installation von Sal...,"[intel, ethernet, hardware aufgebaut werden, h...",NaN
6,5,40,5_build exception_build_project build exceptio...,Brennvorgänge mit Build-Exception-Fehlern,"[build exception, build, project build excepti...","[fehlermeldung wmem, build exception, build ex...",[Brennvorgänge mit Build-Exception-Fehlern\n\n...,"[project build, an adresse, fehlermeldung wmem...",NaN


In [ ]:
doc = df['text'].iloc[106]
print(doc)
topics, probs = topic_model.transform([doc])

print("Predicted topic ID:", topics[0])
print("Custom label:", topic_model.custom_labels_[topics[0]])
print("Topic probability:", probs[0])
print("Representative words for this topic:", topic_model.get_topic(topics[0]))

 Ich bekomme immerwieder den Fehlercode 9. Neuinstallation und Anwendung von whrescue hilft nicht. Jetzt ist die maximale Anzahl von Freischaltungen erreicht und ich komme gar nicht mehr weiter. Fuer einen 65 jaehrigen Berufstaetigen und langjaehrigen guten Warehousekunden schon eine sehr aergerliche Sache. Bitte helfen Sie mir. MfG


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2025-08-27 14:11:36,337 - BERTopic - Predicting topic assignments through cosine similarity of topic and document embeddings.


Predicted topic ID: 2
Custom label: Warehouses Prozessverwaltung und -übertragung
Topic probability: [0.97546273 0.97661376 0.97308934 0.9898667  0.97142255 0.9697744
 0.9646319 ]
Representative words for this topic: [['fuer', 0.08552468292571298], ['moechte', 0.03477227544107411], ['ueber', 0.033421885447622784], ['freundlichen gruessen', 0.03143450553001174], ['mit freundlichen gruessen', 0.03143450553001174], ['gruessen', 0.03102304387285357], ['laeuft', 0.027339098427996693], ['moeglich', 0.026884313238172124], ['koennen', 0.026884313238172124], ['abrechnung', 0.023181516960716075]]
